In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini: Um Overview de cenários multimodais

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Executar no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Ver no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abrir no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão Geral

### Gemini

Gemini é uma família de modelos generativos de IA desenvolvidos pelo Google DeepMind e projetados para casos de uso multimodais. A API Gemini dá acesso aos modelos Gemini Pro Vision e Gemini Pro.

### API Vertex AI Gemini

A API Vertex AI Gemini fornece uma interface unificada para interagir com modelos Gemini. Atualmente existem dois modelos disponíveis na API Gemini:

- **Modelo Gemini Pro** (`gemini-pro`): Projetado para lidar com tarefas de linguagem natural, bate-papo multivoltas de texto e código e geração de código.
- **Modelo Gemini Pro Vision** (`gemini-pro-vision`): Suporta prompts multimodais. Você pode incluir texto, imagens e vídeo em suas solicitações de prompt e obter respostas em texto ou código.

Você pode interagir com a API Gemini usando os seguintes métodos:

- Use o [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para testes rápidos e geração de comandos
- Use o SDK da Vertex AI

Este notebook se concentra no uso do **Vertex AI SDK para Python** para chamar a API Vertex AI Gemini.

Para obter mais informações, consulte a documentação [IA Generativa na Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Este notebook demonstra uma variedade de casos de uso multimodais para os quais o Gemini pode ser usado.

#### Casos de uso multimodais

Em comparação com LLMs somente texto, a multimodalidade do Gemini Pro Vision pode ser usada para muitos novos casos de uso:

Exemplos de casos de uso com **texto e imagem(s)** como entrada:

- Detectando objetos em fotos
- Compreender telas e interfaces
- Compreensão de desenho e abstração
- Compreender gráficos e diagramas
- Recomendação de imagens com base nas preferências do usuário
- Comparar imagens em busca de semelhanças, anomalias ou diferenças

Exemplos de casos de uso com **texto e vídeo** como entrada:

- Gerando uma descrição de vídeo
- Extração de tags de objetos ao longo de um vídeo
- Extração de destaques/mensagens de um vídeo

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.


## Primeiros passos


### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe as bibliotecas necessárias


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

## Importe o modelo `Gemini 1.0 Pro Vision`

Gemini Pro Vision (`gemini-1.0-pro-vision`) é um modelo multimodal que suporta prompts multimodais. Você pode incluir texto, imagem(s) e vídeo em suas solicitações de prompt e obter respostas em texto ou código.

In [ ]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### Defina algumas funções auxiliares

Defina funções auxiliares para carregar e exibir imagens.

In [ ]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

## Compreensão de contexto em múltiplas imagens

Uma das capacidades do Gemini é ser capaz de raciocinar através de múltiplas imagens.

Este é um exemplo de uso do Gemini para calcular o custo total de mantimentos usando uma imagem de frutas e uma lista de preços:

In [ ]:
image_grocery_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/banana-apple.jpg"
image_prices_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/pricelist.jpg"
image_grocery = load_image_from_url(image_grocery_url)
image_prices = load_image_from_url(image_prices_url)

instructions = "Instruções: Considere a imagem a seguir que contem frutas:"
prompt1 = "Quanto eu pagarei portas frutas considerando esta tabela de preços?"
prompt2 = """
Responda à pergunta através destas etapas:
Passo 1: Identifique que tipo de fruta existe na primeira imagem.
Passo 2: Conte a quantidade de cada fruta.
Passo 3: Para cada item da primeira imagem, verifique o preço do item na tabela de preços.
Passo 4: Calcule o preço subtotal de cada tipo de fruta.
Passo 5: Calcule o preço total das frutas usando os subtotais.
Passo 6: se eu pagar com uma nota de 20 reais, quanto receberei de troco?

Responda e descreva as etapas realizadas:"""

contents = [
    instructions,
    image_grocery,
    prompt1,
    image_prices,
    prompt2,
]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Compreensão de telas e interfaces

O Gemini também pode extrair informações de telas de dispositivos, interfaces de usuário, capturas de tela, ícones e layouts.

Por exemplo, se você inserir a imagem de um fogão, poderá pedir ao Gemini que forneça instruções para ajudar um usuário a navegar na interface e responder em diferentes idiomas:

In [ ]:
image_stove_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/stove.jpg"
image_stove = load_image_from_url(image_stove_url)

prompt = """Como posso acertar o relógio deste aparelho?
Se as instruções incluírem botões, explique também onde esses botões estão fisicamente localizados.
Forneça três versões de intruções: em português, em inglês e francês.
"""

contents = [image_stove, prompt]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

**Nota:** A resposta pode não ser totalmente precisa, pois o modelo pode ter alucinações; entretanto, o modelo é capaz de identificar a localização dos botões e traduzir em uma única consulta. Para mitigar as alucinações, uma abordagem é fundamentar o LLM com técnicas como RAG, o que está fora do escopo deste notebook.

## Noções básicas sobre relacionamentos de entidades em diagramas técnicos

O Gemini possui recursos multimodais que permitem compreender diagramas e executar etapas práticas, como otimização ou geração de código. Este exemplo demonstra como o Gemini pode decifrar um diagrama de relacionamento de entidade (ER), entender os relacionamentos entre tabelas, identificar requisitos para otimização em um ambiente específico como o BigQuery e até mesmo gerar o código correspondente.

In [ ]:
image_er_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/er.png"
image_er = load_image_from_url(image_er_url)

prompt = "Documente as entidades e relacionamentos neste diagrama entidade relacionamento (DER)."

contents = [prompt, image_er]

# Use a more deterministic configuration with a low temperature
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    candidate_count=1,
    max_output_tokens=2048,
)

responses = multimodal_model.generate_content(
    contents,
    generation_config=generation_config,
    stream=True,
)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Recomendações baseadas em múltiplas imagens

Gemini é capaz de comparar imagens e fornecer recomendações. Isso pode ser útil em setores como ecommece e varejo.

Abaixo está um exemplo de escolha de qual par de óculos seria mais adequado para um rosto oval:

In [ ]:
image_glasses1_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/glasses1.jpg"
image_glasses2_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/glasses2.jpg"
image_glasses1 = load_image_from_url(image_glasses1_url)
image_glasses2 = load_image_from_url(image_glasses2_url)

prompt1 = """
Qual desses óculos você me recomenda com base no formato do meu rosto?
Eu tenho um rosto oval.
----
Óculos 1:
"""
prompt2 = """
----
Óculos 2:
"""
prompt3 = """
----
Explique como você toma essa decisão.
Forneça sua recomendação com base no formato do meu rosto e no raciocínio para cada um.
Forneça a resposta em formato JSON.
"""

contents = [prompt1, image_glasses1, prompt2, image_glasses2, prompt3]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Semelhanças e Diferenças

Os modelos Gemini podem comparar imagens e identificar semelhanças ou diferenças entre objetos.

O exemplo a seguir mostra duas cenas da Marienplatz em Munique, Alemanha, que são ligeiramente diferentes. O Gemini pode comparar as imagens e encontrar semelhanças/diferenças:

In [ ]:
image_landmark1_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/landmark1.jpg"
image_landmark2_url = "https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/landmark2.jpg"
image_landmark1 = load_image_from_url(image_landmark1_url)
image_landmark2 = load_image_from_url(image_landmark2_url)

prompt1 = """
Considere as seguintes imagens:
Imagem 1:
"""
prompt2 = """
Imagem 2:
"""
prompt3 = """
Responda as seguintes perguntas em uma frase curta.

1. O que é mostrado na Imagem 1?
2. O que há de semelhante entre as duas imagens?
3. Qual a diferença entre a Imagem 1 e a Imagem 2 em termos de conteúdo ou pessoas mostradas?"""

contents = [prompt1, image_landmark1, prompt2, image_landmark2, prompt3]

generation_config = GenerationConfig(
    temperature=0.0,
    top_p=0.2,
    top_k=40,
    candidate_count=1,
    max_output_tokens=2048,
)

responses = multimodal_model.generate_content(
    contents,
    generation_config=generation_config,
    stream=True,
)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Gerando uma descrição de vídeo

O Gemini também pode extrair tags de um vídeo:

> Vídeo: https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4

In [ ]:
prompt = """
O que é mostrado neste vídeo?
Onde devo ir para ver isso?
Quais são 5 outros lugares do mundo com esta aparência?
"""

video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

> Você pode confirmar que o local é realmente Antalya, Turquia, visitando a página da Wikipedia: https://en.wikipedia.org/wiki/Antalya

## Extraindo tags de objetos ao longo do vídeo

O Gemini também pode extrair tags de um vídeo.

> Vídeo: https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/photography.mp4

In [ ]:
prompt = """
Responda às seguintes perguntas usando apenas o vídeo:
- O que está no vídeo?
- Qual é a ação no vídeo?
- Fornecer as 10 melhores tags para este vídeo?
"""

video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/photography.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Fazendo mais perguntas sobre um vídeo

Abaixo está outro exemplo para uso com o Gemini neste cenário de perguntas sobre um vídeo.

> Vídeo: https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4
> _Observação: embora este vídeo contenha áudio, o Gemini atualmente não suporta entrada de áudio e só responderá com base no vídeo._

In [ ]:
prompt = """
Responda às seguintes perguntas usando apenas o vídeo:
Qual é a profissão da pessoa principal?
Quais são as principais funcionalidades do telefone destacadas?
Em que cidade isso foi gravado?
Forneça a resposta JSON.
"""

video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Recuperando informações extras além do vídeo

> Video: https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/ottawatrain3.mp4


In [ ]:
prompt = """
Que linha é essa?
onde isso vai?
Quais são as estações/paradas?
"""
video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/ottawatrain3.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

> Você pode confirmar que esta é realmente a Linha da Confederação na Wikipedia aqui: https://en.wikipedia.org/wiki/Confederation_Line